In [11]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re

# 크롤링할 기간 설정
start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 2)

# 크롤링할 URL 설정
url = "https://news.daum.net/breakingnews/digital"

# 크롤링할 기사 링크를 저장할 리스트 초기화
links = set()

# 주어진 기간 동안 각 페이지의 기사 링크 추출
while start_date <= end_date:
    
    # 현재 날짜에 해당하는 페이지 URL 생성
    date_str = start_date.strftime("%Y%m%d")
    print(f"\n\n☞☞☞☞☞날짜:  {date_str}")
    
    page_url = f"{url}?regDate={date_str}"
    print(f"☞☞☞☞☞Page_URL:   {page_url}\n\n")
    
    num_page = 1
    
    last_page = False

    # 각 페이지의 기사 링크 추출
    while True:
        page_url_num = f"{page_url}&page={num_page}"
        
        
        # User-Agent 정보 추가
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        res = requests.get(page_url_num, headers=headers)

        # HTTP 요청이 성공했는지 확인
        if res.status_code != 200:
            print(f"Failed to retrieve page: {page_url_num}")
            continue

        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(res.content, "html.parser")

        # robots.txt 파일 참조
        robot_check = soup.find('meta', attrs={'name': 'robots'})
        if robot_check and 'nofollow' in robot_check['content']:
            print('Robots.txt 를 준수하지 않습니다.')
            break

        # 각 기사의 링크를 추출하여 리스트에 추가
        article_links = soup.select(".list_news2 a.link_txt")
#        print(f"\n☞Article_Links:   {article_links}")
        
        if not article_links:
            print(f"\n☞ CSS선택자 .list_news2 a.link_txt 에 해당하는 링크를 찾을 수 없습니다.")
            break

        for link in article_links:
            article_url = link["href"]
            article_res = requests.get(article_url)
            
            # HTTP 요청이 성공했는지? 여부를 확인하기 위한 코드
            if article_res.status_code !=200:
                print(f"HTTP요청이 실패했슴:  {article_url}")
                continue
            
            article_soup = BeautifulSoup(article_res.content, "html.parser")
            date = article_soup.select_one(".info_view .txt_info")
            
            # 작성일자 추출
            
            # date변수가 None 값을 가질 경우 건너뛰도록 아래와 같이 코딩하거나,
            if date:
                date = re.search(r"\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2}", date.get_text())
                if date:
                    date = date.group()
                    date = datetime.strptime(date, "%Y.%m.%d %H:%M")
                    print(f"date:  {date}")
             # 정규표현식을 수정하여 일치하는 항목이 없는 경우 None 대신 빈 문자열을 반환하도록  아래와 같이 코딩
#             date = re.search(r"\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2}", date.get_text() or "")
#             if date:
#                 date = date.group()
#                 date = datetime.strptime(date, "%Y.%m.%d %H:%M")
#                 print(f"date:  {date}")
                

            if (article_url, date):                     
                links.add((article_url, date))
                                

        # 추출된 링크를 이용하여 기사 내용 크롤링        
        for link, date in links:
        
            # 기사 URL 출력 (테스트용)
            print(f"\n\n☞기사링크: {link}")

            # 기사 HTML 코드 가져오기
            res = requests.get(link)

            # HTTP 요청이 성공했는지 확인
            if res.status_code != 200:
                print(f"Failed to retrieve article: {link}")
                continue

            # BeautifulSoup으로 HTML 파싱
            soup = BeautifulSoup(res.content, "html.parser")

            # 기사 제목 추출
            title = soup.select_one(".tit_view").get_text()

            # 기사 본문 추출
            contents = soup.select(".article_view p")
            content = "\n".join([c.get_text().strip() for c in contents])

            # 추출한 제목과 내용 출력
            print(f"☞기사제목: {title}")
            print(f"☞작성일자: {date_str}")
            print(f"☞Page_URL_NUM :   {page_url_num}")
            print(f"☞기사내용: \n{content}")
#            print(f"☞페이지: {num_page}") #page_url_num의 num_page를 확인하고 싶을때 출력
        
        # 마지막 페이지인지? 여부를 검사하여 다음 페이지로 이동
        
        last_page = (len(article_links) <= 14)
            # 한페이지에 15개의 기사가 있으므로 기사의 개수가 14개이하일때 마지막 페이지로 간주하도록 설정
            # IT/경제/문화 모두 한페이지에 15개의 기사가 있다.
        
        if last_page:
            print(f"\n☞ {date}의 마지막 페이지입니다. {num_page}")
            break
        num_page += 1

    # 다음 날짜로 이동
    start_date += timedelta(days=1)




☞☞☞☞☞날짜:  20230201
☞☞☞☞☞Page_URL:   https://news.daum.net/breakingnews/digital?regDate=20230201




☞기사링크: https://v.daum.net/v/20230201210302630
☞기사제목: "혼자 1년에 20만원 내라고? 넷플릭스 그만 봐야겠다" 부글부글
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=1
☞기사내용: 

"이번 달까지만 보고 해지해야겠어요. 어둠의 경로로 봐야 하나…고민되네요."
직장인 김모 씨(36)는 지인 3명과 함께 모두 4명이 넷플릭스 계정을 공유하고 있다. 김 씨가 이용하는 넷플릭스 프리미엄 서비스 월 이용 요금은 1만7000원이지만, 실제 내는 비용은 4명이 나눈 월 4250원이다.
넷플릭스 계정 공유 금지 소식을 접한 그는 "그동안 계정을 공유하며 가성비(가격 대비 성능) 있게 넷플릭스를 봤는데 이제는 해지해야 할 것 같다. 혼자 비용이 연 20만원이면 부담스럽다"고 털어놨다. 넷플릭스의 바뀐 방침에서도 가족끼리는 계정 공유가 가능하지만, 거주지가 달라 '동거 가족'이 아닌 남동생 김 씨 역시 요금을 따로 내야 한다.
"넷플릭스 위해 오프라인 정모 해야 하나"

넷플릭스가 계정 공유 단속을 예고하자 국내 가입자들은 아쉽다는 반응을 쏟아내고 있다. 국내 넷플릭스 이용자 절반 이상이 계정을 공유하고 있다. 대부분 비용을 나눠내고 있어 계정 공유가 금지되면 비용 부담이 2~4배 늘어나는 셈이다. 최근 국내 넷플릭스 공식 홈페이지에서 이같은 내용의 과금 정책이 게시되자 이용자들은 크게 반발하는 분위기다.
1일 업계에 따르면 넷플릭스는 국내 공식 홈페이지를 통해 넷플릭스 계정 공유 단속을 안내했다. 앞서 넷플릭스는 지난해 4분기 실적 발표 이후 공개한 주주 서한에서 2023년도 1분기 말에 계정 공유 유료화를 대대적으로 확대하

☞기사제목: 축사하는 장영진 산업부 1차관
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=1
☞기사내용: 

(서울=연합뉴스) 장영진 산업통상자원부 1차관이 1일 서울 삼성동 코엑스 그랜드인터콘티넨탈호텔에서 세미콘 코리아 2023 부대행사로 열린 '세미콘(SEMICON) 코리아 2023 인더스트리 리더십 디너'에 참석해 축사를 하고 있다. 2023.2.1 [산업통상자원부 제공. 재판매 및 DB 금지]
photo@yna.co.kr
▶제보는 카톡 okjebo


☞기사링크: https://v.daum.net/v/20230201214123278
☞기사제목: 카트 드리프트 "완성형 게임 향한 질주 멈추지 않는다"
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=1
☞기사내용: 

넥슨 레이싱 게임 '카트라이더: 드리프트'가 향후 운영 방향성을 1일 공유했다. 김대겸 해설위원과 조재윤 니트로 스튜디오 카트라이더: 드리프트 디렉터는 라이브 방송으로 2월 16일 업데이트와 3월 9일 정규 시즌 프리뷰를 설명했다.
조 디렉터는 "카트라이더: 드리프트 방향성을 공유하기 위해 이 자리를 준비했다. 많은 내용을 준비했으니 재밌게 시청하길 바란다. e스포츠 관련 내용은 2월 8일 방송으로 소개할 예정이다"며 방송을 시작했다.
2월 16일 업데이트에서는 B2~L3 라이센스 항목 축소, 제거 항목 라이센스 보상 재배치, 라이센스 미션 카트 고정 및 밸런스 조정, 레벨 2 트랙 이용 조건 제거, 라이센스 이용 안내 추가 개선안, 연습 모드 개선 등 다양한 변화가 반영될 예정이다.
라이센스 획득 과정에서 느껴지는 지루함을 줄이고 라이센스 획득 의미를 각인시키기 위한 조치다. 조 디렉터는 "라이센스 관련 피드백이 많았다. 그 의견을 바

☞기사제목: TV조선 재승인 고의감점 의혹 방통위 국장도 구속
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=2
☞기사내용: 

2020년 TV조선 재승인 심사과정에서 고의감점 개입 의혹을 받은 방송통신위원회 양모 국장에 대한 구속영장이 발부됐다.
서울북부지법 임기환 부장판사는 1일 영장실질심사를 한 뒤 양 국장에 대한 구속영장을 발부했다. 도망의 염려와 증거인멸의 우려가 있다고 발부 이유를 밝혔다.
앞서 지난달 11일 영장실질심사 당시에는 현재 구속해야 할 사유와 상당성을 인정하기 어렵고 방어권을 보장해야 한다며 구속영장을 기각했다. 이후 검찰이 보강수사를 진행, 지난달 29일 양 국장 구속영장을 재청구했고 사흘 만인 이날 구속영장이 발부됐다. 양 국장은 재승인 심사과정에서 심사위원에 평가점수를 알려주고 점수표 수정을 요구한 혐의를 받는 것으로 알려졌다.
양 국장과 함께 지난달 11일 영장실질심사를 받은 차모 과장은 당일 구속영장이 발부됐다. 재판부는 중요 혐의사실에 대한 소명이 있고 감사·수사단계에서 태도 등에 비춰 증거인멸 염려가 있다고 판단했다. 차 과장은 지난달 26일 법원에 구속적부심을 청구했다. 법원은 청구 이유가 없다며 기각했고 이후 위계공무집행방해죄 혐의로 구속 기소됐다.
앞서 서울북부지검 형사5부는 2020년 종합편성채널 재승인 심사 당시 TV조선 재승인 심사·평가업무를 담당한 방통위 국·과장을 상대로 직권남용, 위계에 의한 공무집행방해, 공무상 비밀누설 등 혐의로 구속영장을 청구했다. 방통위 상대 세 차례 압수수색 및 수사에 따른 결정이다.
TV조선 재승인 심사에 개입해 중점 심사항목 점수를 고의로 낮춰 조건부 재승인을 유도하고 재승인 기간을 명확한 근거 없이 1년 단축한 것으로 판단했다.
한편 TV조선 재승인 고의감점에 관여한 의혹으로 지난달 검찰 참고인 조사를 받은 이모 방통위 정책위원은 임기가 종료돼 퇴사하

☞기사제목: 카트 드리프트 "완성형 게임 향한 질주 멈추지 않는다"
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=2
☞기사내용: 

넥슨 레이싱 게임 '카트라이더: 드리프트'가 향후 운영 방향성을 1일 공유했다. 김대겸 해설위원과 조재윤 니트로 스튜디오 카트라이더: 드리프트 디렉터는 라이브 방송으로 2월 16일 업데이트와 3월 9일 정규 시즌 프리뷰를 설명했다.
조 디렉터는 "카트라이더: 드리프트 방향성을 공유하기 위해 이 자리를 준비했다. 많은 내용을 준비했으니 재밌게 시청하길 바란다. e스포츠 관련 내용은 2월 8일 방송으로 소개할 예정이다"며 방송을 시작했다.
2월 16일 업데이트에서는 B2~L3 라이센스 항목 축소, 제거 항목 라이센스 보상 재배치, 라이센스 미션 카트 고정 및 밸런스 조정, 레벨 2 트랙 이용 조건 제거, 라이센스 이용 안내 추가 개선안, 연습 모드 개선 등 다양한 변화가 반영될 예정이다.
라이센스 획득 과정에서 느껴지는 지루함을 줄이고 라이센스 획득 의미를 각인시키기 위한 조치다. 조 디렉터는 "라이센스 관련 피드백이 많았다. 그 의견을 바라보는 것이 아닌 충분히 검토했다. 프리시즌 피드백은 서비스 방향을 확립하는 시기다. 계속 수정할 계획이다"고 전했다.

연습 모드 관련해선 PVP 플레이만 가능한 옵션이 추가되고 연습 모드 옵션 안내 가이드를 추가한다. 초보자의 경우 AI 매칭이 고정적으로 설정되어 있다. 이는 자유롭게 변경할 수 있다.
레이서명은 최초 생성 시 재확인 후 생성 가능하도록 개선된다. 최근 에이스, 광산 수레가 다른 카트에 비해 성능이 좋은 상황을 해소하기 위해 상향 평준화로 1차 조정이 이뤄질 예정이다.
레이서명 변경은 최초 1회 무료로 제공한다. 이후 K 코인으로 구매할 수 있다. 변경 시 15일 동안 다른 닉네임으로 변경할 수 없다. 이는 비매너 유저가 활개치는 상황을 막기 위한 

☞기사제목: 슈나이더 일렉트릭-한솔 PNS, 스마트팩토리 사업 협력
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=2
☞기사내용: 
(지디넷코리아=남혁우 기자)슈나이더 일렉트릭 코리아(대표 김경록)가 한솔 PNS(대표 김형준)와 ‘스마트 팩토리 사업 추진’을 위한 업무협약(MOU)’을 체결했다고 1일 밝혔다.
서울 강서구 슈나이더 일렉트릭 코리아 사무실에서 열린 협약식에는 슈나이더 일렉트릭 코리아 김경록 대표, 서상훈 본부장과 한솔PNS 김형준 대표 등 양사 주요 경영진이 참석했다.
양사는 한솔 PNS의 IT 역량과 슈나이더 일렉트릭 OT 영역의 강점을 결합해 스마트팩토리 분야에서 상호 이익과 발전을 위해 운영 효율성을 높이고, 에너지 관리를 위한 비즈니스 모델을 구축한다는 공동의 목표를 세웠다.

한솔 PNS는 2021년 자사 스마트 팩토리 전문회사 한솔코에버를 인수한 후 MES, EMS, 예지보전 서비스, 머신 러닝 서비스 등의 전문성을 보유하고, 다양한 제품 라인업을 갖추고 있어 계열사 외에도 산업별로 특화된 고객에게 IT 솔루션과 구축 서비스를 제공하고 있다.
슈나이더 일렉트릭과 한솔 PNS는 ▲제조실행시스템(MES)의 상호 솔루션 협력, ▲에너지 관리 시스템(EMS) 연계, ▲스마트팩토리 고도화 프로젝트 진행 등 각 분야에서 협력을 이어간다.
한솔PNS는 이번 협약으로 아비바(AVEVA)와 프로레이트(ProLeiT)의 제조실행시스템(MES)을 도입해 상호 솔루션 연계 가능성을 검토해 솔루션의 안정성을 확보하고, 신규 세그먼트 업체에 맞는 최적화된 시스템을 구축할 것으로 기대한다.
슈나이더 일렉트릭 에너지 관리 솔루션(EMS)은 센서, 무선미터기 등 하드웨어뿐만 아니라, 데이터 가시화 및 리포트를 제공하는 소프트웨어를 포함한다. 더불어 화재감시 솔루션까지 갖추고 있어 고도화를 요하는 고객에게 추가 분석 서비스도 제공한다.
양

☞기사제목: "혼자 1년에 20만원 내라고? 넷플릭스 그만 봐야겠다" 부글부글
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=2
☞기사내용: 

"이번 달까지만 보고 해지해야겠어요. 어둠의 경로로 봐야 하나…고민되네요."
직장인 김모 씨(36)는 지인 3명과 함께 모두 4명이 넷플릭스 계정을 공유하고 있다. 김 씨가 이용하는 넷플릭스 프리미엄 서비스 월 이용 요금은 1만7000원이지만, 실제 내는 비용은 4명이 나눈 월 4250원이다.
넷플릭스 계정 공유 금지 소식을 접한 그는 "그동안 계정을 공유하며 가성비(가격 대비 성능) 있게 넷플릭스를 봤는데 이제는 해지해야 할 것 같다. 혼자 비용이 연 20만원이면 부담스럽다"고 털어놨다. 넷플릭스의 바뀐 방침에서도 가족끼리는 계정 공유가 가능하지만, 거주지가 달라 '동거 가족'이 아닌 남동생 김 씨 역시 요금을 따로 내야 한다.
"넷플릭스 위해 오프라인 정모 해야 하나"

넷플릭스가 계정 공유 단속을 예고하자 국내 가입자들은 아쉽다는 반응을 쏟아내고 있다. 국내 넷플릭스 이용자 절반 이상이 계정을 공유하고 있다. 대부분 비용을 나눠내고 있어 계정 공유가 금지되면 비용 부담이 2~4배 늘어나는 셈이다. 최근 국내 넷플릭스 공식 홈페이지에서 이같은 내용의 과금 정책이 게시되자 이용자들은 크게 반발하는 분위기다.
1일 업계에 따르면 넷플릭스는 국내 공식 홈페이지를 통해 넷플릭스 계정 공유 단속을 안내했다. 앞서 넷플릭스는 지난해 4분기 실적 발표 이후 공개한 주주 서한에서 2023년도 1분기 말에 계정 공유 유료화를 대대적으로 확대하겠다고 밝힌 바 있다. 홈페이지 고객센터 페이지에 따르면 원칙적으로 한 가구 내에 함께 살지 않은 사람은 계정 공유가 금지된다. '동거 가족'이 아니라고 판단되면 인증 요청을 보내 시청이 차단된다. 인증은 15분 내 이뤄져야 한다.
넷플릭스는 "회원의 가구 구성원이



☞기사링크: https://v.daum.net/v/20230201182408332
☞기사제목: NASA 무인 우주선 발사에 매스웍스 기술 활용
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=3
☞기사내용: 

“나사는 아르테미스 프로젝트에서 혁신적인 기술을 통해 달 표면을 자세히 탐사하고 최초로 여성과 유색 인종 우주인이 달에 착륙하도록 할 것입니다. 또한 달에서 장기간 체류하며 배운 것들을 통해 궁극적으로 화성에 최초로 우주인을 보낼 것입니다.” 아르테미스 프로젝트의 임무를 시작한 미 항공우주국 나사(NASA)가 이같이 강조했다.
1972년을 마지막으로 인류는 달에 발을 딛지 않았다. 하지만 나사는 인류를 다시 우주로 보내기 위해 그리스 신화에서 사냥과 달의 여신의 이름을 딴 아르테미스 프로젝트를 추진하고 있다. 사람이 탑승하지 않는 ‘아르테미스 1호’ 임무는 달까지 가 ‘우주발사시스템(SLS)’이라는 신형 로켓을 테스트하는 것이다. 우주발사시스템(SLS)은 지금까지 제작된 로켓 중 가장 강력한 로켓으로서 39.1 메가 뉴턴(880만 파운드)이라는 막대한 추력을 자랑한다.
하지만 우주발사시스템(SLS)만이 아르테미스의 전부는 아니다. 30층 높이에 달하는 이 로켓의 상단에는 최대 6명까지 태울 수 있는 유인 우주선인 오리온(Orion)이 탑재됐으며, 심우주의 극한 조건에서 탑승객을 보호하도록 설계되었다. 우주발사시스템(SLS)은 이 오리온을 달로 향하는 궤도에 올려놓는 역할을 한다.
무인 비행 ‘아르테미스 1호’ 임무는 달을 탐사하고 미래에 달 표면에 베이스캠프를 짓는 장기적 목표의 첫걸음이다. 일 년 뒤 오리온 기체에 우주인이 승선하는 ‘아르테미스 2호’는 같은 여정을 떠나게 되며, 2024년의 ‘아르테미스 3호’ 임무를 통해 인류 최초로 여성과 유색 인종 우주비행사가 달의 남극을 밟게 된다. 나사는 아르테미스 임무를 통해 

☞기사제목: 경기도사회서비스원, ‘창립 3주년’ 사회공헌활동 나서
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=3
☞기사내용: 


경기도사회서비스원(원장 안혜영)이 창립 3주년을 맞아 기념식과 사회공헌활동을 진행했다.
경기도사회서비스원은 온라인 창립 3주년 기념 행사를 개최했다고 1일 밝혔다. 행사는 원장 인사말과 직원 대표의 고객서비스 헌장 선언, ESG경영 실천 다짐, 단체 사진 촬영 등 지난 활동을 격려하고 새로운 도약을 다짐하기 위해 마련됐다.
기념식에 이어 안혜영 경기도사회서비스원장과 직원 60여명은 평택시 내리문화공원 일대에서 사회공헌활동을 진행했다. 환경부가 지정한 생태교란 식물을 제거하고, 겨울 식물 데이터 수집과 산책이나 조깅을 하며 쓰레기를 줍는 ‘플로깅’을 통해 ESG 경영 차원의 창립 행사에 의미를 더했다.
안혜영 원장은 “경기도사회서비스원은 설립 이래 24개 소속시설 300여명의 종사자와 함께 공공기관 종사자로서 추구해야할 사회적 가치와 책무를 다하기 위해 노력해왔다”며 “2023년에는 보다 좀 더 세밀하고 촘촘한 복지서비스 제공을 위해 노력하겠다. 더 나아가 생활밀착형 원스톱 복지 서비스를 구현하기 위해 종합적 복지정책을 수립하고 전 도민의 전 생애 주기에 걸친 통합복지정책을 도출하는데 선도적 역할을 수행하겠다”고 다짐했다.
한편 경기도사회서비스원은 2일 ‘경기도사회서비스원의 역할과 발전 방향’을 주제로 각계각층의 전문가를 초빙해 운영에 대한 다양한 의견을 듣는 ‘해커톤 토론’을 개최한다.
손사라 기자 sarah@kyeonggi.com


☞기사링크: https://v.daum.net/v/20230201202242954
☞기사제목: SK실트론, 구미 반도체 공장에 2조3000억원 투자
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnew

☞기사제목: 아크서브, UDP 9.0 발표…오라클·MS DB 지원 확대
☞작성일자: 20230201
☞Page_URL_NUM :   https://news.daum.net/breakingnews/digital?regDate=20230201&page=3
☞기사내용: 

아크서브는 자사 중앙집중식 백업 및 재해복구 솔루션 'UDP'의 9.0버전을 선보인다고 1일 밝혔다.
아크서브에 따르면 UDP 9.0은 소포스(Sophos) 솔루션을 적용한 사이버 보안, 데이터 위변조 방지 스토리지, 테이프 백업 등 기능을 제공한다. 오라클 RMAN을 사용해 백업된 테라바이트 규모의 오라클PDB(Pluggable DB)를 신속하게 복구, 모든 테이블스페이스 및 제어 파일 등을 원래 위치에 복원할 수 있도록 해준다. MS(마이크로소프트) SQL서버 배포를 위한 고급 복원 옵션도 지원한다. 두 복구 지점 간 모든 트랜잭션 지점으로 DB를 복구할 수 있다. 일관성 검사를 실행해 검색된 데이터가 사용 가능한 데이터인지 확인해 복원 무결성을 보장한다.
아크서브 UDP 9.0은 클라우드 객체 스토리지를 통해 가용성, 내구성, 확장성을 보장한다. 중복 제거된 백업본을 AWS(아마존웹서비스) S3, 와사비(Wasabi) 또는 구글 클라우드 스토리지에 직접 저장함으로써 총 소유 비용을 절감하고 재해 복구 지원을 강화할 수 있다는 게 회사의 설명이다. 멀티테넌시 기능과 안전한 ID 관리 기능을 갖춘 클라우드 기반 관리 콘솔도 제공한다.
플로리안 말레키(Florian Malecki) 아크서브 마케팅 담당 부사장은 "UDP 9.0은 데이터 복원력을 한 차원 높여 향상된 데이터 보호, 성능 안정성 및 관리 기능을 제공한다. 특히 하나의 솔루션으로 랜섬웨어에 대응할 수 있도록 해준다"고 말했다.팽동현기자 dhp@dt.co.kr


☞기사링크: https://v.daum.net/v/20230201192217444
☞기사제목: [자막뉴스] 빙하기 이후 처음...국내 밤하늘에 나타난 건
☞작성일자: 20230

KeyboardInterrupt: 